In [1]:
!pip install --quiet -r requirements.txt

In [2]:
!nvidia-smi
#!pip list

Thu Nov 14 06:07:14 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060        Off |   00000000:01:00.0  On |                  N/A |
|  0%   43C    P0             42W /  170W |    2568MiB /  12288MiB |     10%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import warnings
warnings.filterwarnings('ignore')

import os, gc
import numpy as np
import pandas as pd

from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from joblib import dump, load
import datatable as dtable
#from mlxtend.evaluate import GroupTimeSeriesSplit
#import kaggle_evaluation.jane_street_inference_server

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
TEST=True

In [4]:
# check that torch is working and sees the GPU
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

2.4.1+cu121
True
NVIDIA GeForce RTX 3060


In [5]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [6]:
batch_size=64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [8]:
# get cpu, gpu or mps device for training
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return(logits)

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            print(f'type of X is {type(X)}')
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}|{size:>5d}]")
            #cuda_time = utilization(device)
            #print(f"cuda time: {cuda_time:>1d}")

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------------")
    train(train_dataloader,  model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-----------------------
type of X is <class 'torch.Tensor'>
loss: 2.319016 [   64|60000]
type of X is <class 'torch.Tensor'>
loss: 0.558900 [ 6464|60000]
type of X is <class 'torch.Tensor'>
loss: 0.386092 [12864|60000]
type of X is <class 'torch.Tensor'>
loss: 0.502590 [19264|60000]
type of X is <class 'torch.Tensor'>
loss: 0.496098 [25664|60000]
type of X is <class 'torch.Tensor'>
loss: 0.425982 [32064|60000]
type of X is <class 'torch.Tensor'>
loss: 0.375690 [38464|60000]
type of X is <class 'torch.Tensor'>
loss: 0.523975 [44864|60000]
type of X is <class 'torch.Tensor'>
loss: 0.518917 [51264|60000]
type of X is <class 'torch.Tensor'>
loss: 0.566473 [57664|60000]
Test Error: 
 Accuracy: 84.1%, Avg loss: 0.432491 

Epoch 2
-----------------------
type of X is <class 'torch.Tensor'>
loss: 0.278173 [   64|60000]
type of X is <class 'torch.Tensor'>
loss: 0.358616 [ 6464|60000]
type of X is <class 'torch.Tensor'>
loss: 0.260925 [12864|60000]
type of X is <class 'torch.Tensor'>
los

In [14]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [13]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0085, -0.0899, -0.0896,  ..., -0.0081, -0.0081, -0.0084],
        [-0.0272, -0.0478, -0.0719,  ..., -0.0275, -0.0422, -0.0462]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.1853, 0.1406], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0731,  0.1297,  0.3325,  ..., -0.1099, -0.0370, -0.0824],
        [-0.0622, -0.0459, -0.0583,  ..., -0.0466,  0.0796, -0.1867]],
       device='cuda:0', grad_fn=<Slic

In [16]:
lmodel = NeuralNetwork().to(device)
lmodel.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [17]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

lmodel.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = lmodel(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


In [18]:
pred[0].argmax(0)

tensor(9, device='cuda:0')